<a href="https://colab.research.google.com/github/vaidehiu/GenAI-Usecase-Doc-Based/blob/main/IngestingDocUsecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./content/')

!cd '/content/'
#!ls -l
!pip install PdfReader
!pip install langchain
!pip install PyPDF2
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install faiss
!pip install faiss-gpu

from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
import os
from langchain.prompts.prompt import PromptTemplate

Mounted at ./content/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.4 MB/s eta 0:00:00
  Created wheel for PdfReader: filename=pdfreader-0.1.12-py3-none-any.whl size=134539 sha256=d0f1b931bfedf86835a489d81ca07ded86502402a2ce7a23ccbd342af8283fdd
  Stored in directory: /root/.cache/pip/wheels/92/7d/f1/8b8a503d8b444a82e4c3c8b1230c02316c7579b90542de4495
Successfully built PdfReader
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:0

In [ ]:
!ls /content/content/MyDrive/new_diseases_data.pdf

/content/content/MyDrive/new_diseases_data.pdf


In [ ]:
def retrieval_qa_chain(db, return_source_documents):
    llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature": 0.6, "max_length": 500, "max_new_tokens": 700})
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type='stuff',
                                           retriever=db,
                                           return_source_documents=return_source_documents,
                                           )
    return qa_chain

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
## extracting text from pdf files
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

## creating overlapping text chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

## creating embeddings for chunks of text
def get_vectorstore(text_chunks):
    #embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [ ]:
path_to_pdf = ['/content/content/MyDrive/new_diseases_data.pdf']

In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    """
    Extracts all text from a PDF file.

    Parameters:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Example usage:
# Replace 'your_pdf_file.pdf' with the actual filename of the PDF you want to use.
pdf_path = '/content/content/MyDrive/new_diseases_data.pdf'
pdf_text = extract_text_from_pdf(pdf_path)

In [ ]:
raw_text = get_pdf_text(path_to_pdf)

# get the text chunks
text_chunks = get_text_chunks(raw_text)

# create vector store
vectorstore = get_vectorstore(text_chunks)

## creating a db with similarity search and obtaining top 3 most matched vectors of all the vectors present in vector index
db = vectorstore.as_retriever(search_kwargs={'k': 3})


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
bot = retrieval_qa_chain(db,True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
query = "what is Nampdicta ?"
sol=bot(query)

In [ ]:
print(sol['result'])


Yashas is a fictional character from the Ramtronephiach Oculosishore series.


In [ ]:
print(sol['source_documents'])

[Document(page_content="**Disease\nName:**\nNampdicta\n**Description:**\nNampdicta\nis\na\nhighly\nvirulent\nand\nlethal\ninfectious\ndisease\nthat\nhas\nstruck\nfear\ninto\nglobal\nhealth\norganizations\nand\npopulations\nalike.\nCharacterized\nby\nits\nrapid\nonset,\nelusive\nnature,\nand\ndevastating\nimpact,\nNampdicta\nhas\nemerged\nas\na\nsignificant\nthreat\nto\npublic\nhealth.\nThe\ndisease\nis\ncaused\nby\na\nyet-to-be-identified\npathogen,\nsuspected\nto\nbe\na\nmutated\nvirus,\nwith\nthe\nability\nto\nswiftly\nevade\nthe\nimmune\nsystem\nand\ntarget\nmultiple\norgan\nsystems.\nNampdicta's\nmode\nof\ntransmission\nremains\nperplexing,\nas\nit\nappears\nto\nspread\nthrough\nvarious\nvectors,\nincluding\nrespiratory\ndroplets,\nbodily\nfluids,\nand\neven\nfomites.\nThis\ncomplexity\nmakes\ncontainment\nand\nprevention\nstrategies\nexceptionally\nchallenging\nto\ndevelop.\nNampdicta's\nincubation\nperiod\nis\nremarkably\nshort,\nranging\nfrom\nhours\nto\na\nfew\ndays,\nduring\nw

In [ ]:
# normal falcon without context
llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.7,"max_length":500,"max_new_tokens":700})
llm(query)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


"\nI'm sorry, I cannot find any information on Nampdicta. It is possible that it is a made-up term or a specific term used in a particular context. Can you provide more context or information about Nampdicta?"

In [ ]:
ques = ['what are the origins of Numpalofich Legatrosis', ...]

In [ ]:
sol = bot(ques[0])
print(ques[0])
print(sol['result'])

what are the origins of Numpalofich Legatrosis
 The origins of Numpalofich Legatrosis are still unknown, but it is believed to have a complex genetic background.
